<a href="https://colab.research.google.com/github/Profe-Jose-Burgos/Multi-Bot/blob/main/chat_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import json

In [20]:
data = open("/content/drive/MyDrive/hackathon/tryouts.json").read()
intents = json.loads(data)

In [31]:
import nltk
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('wordnet')
from nltk import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [32]:
words = []
classes = []
documents = []
ignore_words = ['?','!']

In [33]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
      x = nltk.word_tokenize(pattern)
      words.extend(x)
      documents.append((x, intent['tag']))

      if intent['tag'] not in classes:
        classes.append(intent['tag'])

words = [lemmatizer.lemmatize(x.lower()) for x in words if x not in ignore_words]

In [34]:
import pickle

In [35]:
pickle.dump(words, open('words.pkl','wb'))
pickle.dump(classes, open('classes.pkl','wb'))


training = []
output_empty = [0] * len(classes)

for doc in documents:   
    count = []    
    pattern_words = doc[0]    
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    for w in words:
        count.append(1) if w in pattern_words else count.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([count, output_row])

In [36]:
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import SGD